<a href="https://colab.research.google.com/github/shawnk1188/ML/blob/main/Product_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Welcome to the Notebook**

### Task 1 - Set up the project

Installing the needed modules.

In [33]:
!pip install openai==1.16.2 python-dotenv pyspark

In [34]:
!pip install ollama

Imporint the modules

In [35]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType

from pyspark.ml.feature import VectorAssembler, PCA
from pyspark.ml.clustering import KMeans
import plotly.express as px
import json
import requests
import ollama
from pyspark.sql import Row
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import ArrayType, FloatType
from pyspark.sql.functions import split, regexp_replace
from functools import reduce



Install colab=xterm

In [36]:
!pip install colab-xterm
%load_ext colabxterm

The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


In [50]:
%xterm

Launching Xterm...

<IPython.core.display.Javascript object>

Install Ollama

In [38]:
!ollama

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model from a Modelfile
  show        Show information for a model
  run         Run a model
  stop        Stop a running model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.


In [39]:
! ollama pull llama3:8b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕▏  12 KB                         
pulling 8ab4849b038c... 100% ▕▏  254 B                         
pulling 577073ffcc6c... 100% ▕▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


Create a Spark session

In [40]:
spark = SparkSession.builder.appName("Product_Recommendation_System").getOrCreate()

Loading the dataset

In [41]:
file_path = "products_dataset.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True,samplingRatio=1)
df.show()

+----------+--------------------+--------------------+
|product_id|               title|         description|
+----------+--------------------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|
|        P1|Turmode 30 ft. RP...|If you need more ...|
|        P2|Large Tapestry Bo...|Polyester cover r...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|
|        P9|Traditional Silve...|This transitional...|
|       P10|15 in. x 59 in. O...|Its easy to add a...|
|       P11|1 qt. #350F-7 Wil...|BEHR PREMIUM PLUS...|
|       P12|Anthracite Cordle...|BlindsAvenue ligh...|
|       P13|SlimGrip 78-Inch ...|Luverne SlimGrip ...|
|       P14|6 in. x 28 in. x ...|Our Rustic Collec...|
|       P1

List of 8 products recently viewed by the user.

In [42]:
recently_viewed_products = [
    'P316',
    'P333',
    'P1115',
    'P1691',
    'P1082',
    'P397',
    'P1441',
    'P1054',
]

### Task 2 - Prepare the dataset

Combine `title` and `description` Columns

In [43]:
df = df.withColumn("combined_text", concat_ws(" ", df.title, df.description))


get the combined_text column and convert it into a list

In [44]:
list_combined_text = df.select("combined_text").rdd.flatMap(lambda x : x).collect()

In [45]:
print(list_combined_text[:4])

["Men's 3X Large Carbon Heather Cotton/Polyester Rain Defender Paxton Heavyweight Hooded Zip-Front Sweatshirt This heavyweight, water-repellent hooded sweatshirt has a zip front for fast layering. ORIGINAL FIT. 13 oz., 75% cotton/25% polyester blend with Rain Defender durable water repellent. Attached, jersey-lined three-piece hood with drawcord closure. Antique-finish brass front zipper. Two front hand-warmer pockets have a hidden security pocket inside. Stretchable, spandex-reinforced rib-knit cuffs and waistband. Locker loop facilitates hanging.", "Turmode 30 ft. RP TNC Female to RP TNC Male Adapter Cable If you need more length between your existing wireless device and Hi-Gain Antenna, this is the product for you. It's compatible with most Wi-Fi Antennas, so it is easy for you to extend your wireless network. Just replace your existing cable that runs between your wireless device and Antenna and you're ready to use your network with extended range.", 'Large Tapestry Bolster Bed Pol

Use OpenAI text embedding model to create the vector embeddings.

In [49]:
!ollama pull mxbai-embed-large


Error: could not connect to ollama app, is it running?


In [51]:
# Specify the model and input text
model_name = "mxbai-embed-large"
text = "This is a sample sentence."
embedding_vectors = []

embedding_vectors = []
for text in list_combined_text:
    response = ollama.embed(model=model_name, input=text)  # Generate embedding for each text
    embedding_vectors.append(response['embeddings'])

print(embedding_vectors[:2])  # Print the first two embedding vectors

[[[-0.005871935, -0.0036916886, 0.016261045, 0.019163456, 0.013635544, 0.04703999, 0.026146324, -0.0072301524, 0.018134994, 0.03361888, 0.019129798, -0.0024371552, 0.0027405259, -0.00727509, -0.03332166, -0.0010470627, -0.026769808, -0.016809832, -0.00055701396, 0.030993236, 0.018969506, 0.027817668, -0.036617443, -0.04319081, -0.044975422, 0.0030430674, -0.018742781, -0.025448555, 0.05072477, 0.07368118, -0.021952286, 0.018112993, 0.02713102, -0.07338533, -0.017991621, 0.00680195, -0.0031158857, -0.022484342, 0.01886529, -0.041809507, 0.01110009, -0.025560498, 0.029895002, -0.030218069, -0.0032993641, -0.035203237, -0.013292233, 0.01576845, 0.013378324, -0.03331853, 0.0134678325, 0.0062352563, 0.047592375, -0.032428853, -0.0053522917, 0.027680475, -0.032236863, 0.032539986, -0.003807818, -0.0016416557, 0.0076306085, -0.020100337, -0.008859784, -0.030249562, 0.007706067, 0.0403519, 0.008023338, 0.052590974, 0.03272819, 0.030929253, 0.032709613, 0.044528037, -0.0055669714, -0.04384269, 

Let't put the embedding vectors into our original dataframe

Convert embedding vectors list into a Pyspark DataFrame

In [52]:
flat_list = reduce(lambda x, y: x + y, embedding_vectors)

In [53]:
features_column_names = [f"embedding_{i}" for i in range(len(flat_list[0]))]

In [54]:
# Convert embedding vectors list into a Pyspark DataFrame
embedding_df = spark.createDataFrame(flat_list, schema=features_column_names)
embedding_df.show()


+------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+-------------+------------+------------+-------------+-------------+-------------+------------+------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+------------+-------------+------------+------------+-------------+-------------+------------+------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+-------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+----

In [55]:
embedding_df = embedding_df.repartition(1).withColumn("row_id", F.monotonically_increasing_id())
embedding_df.show()

+------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+-------------+------------+------------+-------------+-------------+-------------+------------+------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+------------+-------------+------------+------------+-------------+-------------+------------+------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+-------------+------------+------------+------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+----

Add unique `row_id` to each row in our main pyspark dataframe `df`

In [56]:
df = df.repartition(1).withColumn("row_id", F.monotonically_increasing_id())
df.show()

+----------+--------------------+--------------------+--------------------+------+
|product_id|               title|         description|       combined_text|row_id|
+----------+--------------------+--------------------+--------------------+------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|Men's 3X Large Ca...|     0|
|        P1|Turmode 30 ft. RP...|If you need more ...|Turmode 30 ft. RP...|     1|
|        P2|Large Tapestry Bo...|Polyester cover r...|Large Tapestry Bo...|     2|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|16-Gauge-Sinks Ve...|     3|
|        P4|Men's Crazy Horse...|This 9 in. black ...|Men's Crazy Horse...|     4|
|        P5|Mariana 6 ft. Mul...|With robust struc...|Mariana 6 ft. Mul...|     5|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|5 gal. #650C-2 Po...|     6|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|7/8 in. x 4-1/2 i...|     7|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|Ring Gold Bar Car...|     8|
|   

Let's join the two dataframes

In [57]:
df = df.join(embedding_df, on="row_id", how="inner")
df.show()

+------+----------+--------------------+--------------------+--------------------+------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+-------------+------------+------------+-------------+-------------+-------------+------------+------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+------------+-------------+------------+------------+-------------+-------------+------------+------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+-------------+------------+------------+------------+-------------+------

### Task 3 - Cluster products using K-means

Assemble the 512 Embedding Columns into a Single 'features' Column

In [58]:
assembler = VectorAssembler(inputCols=features_column_names, outputCol="features")
data = assembler.transform(df)
data.show()
data = data.select("product_id","title","description" ,"features")
data.show()

+------+----------+--------------------+--------------------+--------------------+------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+-------------+------------+------------+-------------+-------------+-------------+------------+------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+------------+-------------+------------+------------+-------------+-------------+------------+------------+------------+------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------------+-------------+-------------+------------+------------+------------+-------------+------

Apply K-Means Clustering with 5 Clusters on the `features` Column

In [59]:
kmeans = KMeans(k=5, featuresCol='features',predictionCol="cluster")
model = kmeans.fit(data)
clustered_data = model.transform(data)
clustered_data.show()

+----------+--------------------+--------------------+--------------------+-------+
|product_id|               title|         description|            features|cluster|
+----------+--------------------+--------------------+--------------------+-------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|[-0.005871935,-0....|      1|
|        P1|Turmode 30 ft. RP...|If you need more ...|[0.002120536,0.04...|      0|
|        P2|Large Tapestry Bo...|Polyester cover r...|[0.021091936,0.02...|      2|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|[-0.013761591,0.0...|      4|
|        P4|Men's Crazy Horse...|This 9 in. black ...|[0.08196452,0.041...|      0|
|        P5|Mariana 6 ft. Mul...|With robust struc...|[0.019163113,-0.0...|      1|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|[-0.023647875,-0....|      3|
|        P7|7/8 in. x 4-1/2 i...|DEWALT High Perfo...|[8.3629106E-4,0.0...|      0|
|        P8|  Ring Gold Bar Cart|This Ring Bar Car...|[-0.006160132,-0....| 

### Task 4 - Visualize the clusters

Let's reduce the dimensionality of our features for visualization purpose

`512 dimensions => 2 dimensions`

In [60]:
pca = PCA(k=2, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(clustered_data)
pcaresults = pca_model.transform(clustered_data)
pcaresults.show()

+----------+--------------------+--------------------+--------------------+-------+--------------------+
|product_id|               title|         description|            features|cluster|        pca_features|
+----------+--------------------+--------------------+--------------------+-------+--------------------+
|        P0|Men's 3X Large Ca...|This heavyweight,...|[-0.005871935,-0....|      1|[0.06065705504848...|
|        P1|Turmode 30 ft. RP...|If you need more ...|[0.002120536,0.04...|      0|[-0.0767526684017...|
|        P2|Large Tapestry Bo...|Polyester cover r...|[0.021091936,0.02...|      2|[0.06023551032148...|
|        P3|16-Gauge-Sinks Ve...|It features a rec...|[-0.013761591,0.0...|      4|[0.00609054287654...|
|        P4|Men's Crazy Horse...|This 9 in. black ...|[0.08196452,0.041...|      0|[-0.0209181437493...|
|        P5|Mariana 6 ft. Mul...|With robust struc...|[0.019163113,-0.0...|      1|[0.02021657230615...|
|        P6|5 gal. #650C-2 Po...|BEHR PRO i300 Sem...|[

In [61]:
pca_df = pcaresults.select("product_id","pca_features","cluster").toPandas()
pca_df['x'] = pca_df['pca_features'].apply(lambda x: x[0])
pca_df['y'] = pca_df['pca_features'].apply(lambda x: x[1])
pca_df

,product_id,pca_features,cluster,x,y
0,P0,"[0.060657055048481054, 0.008279191381439304]",1,0.060657,0.008279
1,P1,"[-0.07675266840176653, -0.07602536968057429]",0,-0.076753,-0.076025
2,P2,"[0.060235510321488314, 0.2563311214511784]",2,0.060236,0.256331
3,P3,"[0.006090542876544815, 0.006308746668713339]",4,0.006091,0.006309
4,P4,"[-0.02091814374932848, -0.08412996623326116]",0,-0.020918,-0.084130
...,...,...,...,...,...
1995,P1995,"[0.12250833910823078, 0.3448159620113709]",2,0.122508,0.344816
1996,P1996,"[0.030365961640971936, 0.48263264965387176]",2,0.030366,0.482633
1997,P1997,"[0.019002198598417744, 0.030987183420391022]",1,0.019002,0.030987
1998,P1998,"[0.5726998281030653, 0.0713245852188046]",3,0.572700,0.071325


Let's plot the Clusters

In [62]:
def plot_clusters(pca_df, num_clusters=5):
    """
    Plots a 2D visualization of clusters using Plotly Express.

    Parameters:
    - pca_df (DataFrame): A Pandas DataFrame containing columns 'x', 'y', and 'cluster'.
      'x' and 'y' are the 2D PCA components, and 'cluster' indicates the cluster label.
    - num_clusters (int): The number of unique clusters to display.
    - recently_viewed_df (DataFrame, optional): DataFrame with 'x' and 'y' coordinates for recently viewed products.

    This function creates an interactive scatter plot where each point is colored according to its cluster.
    Recently viewed products are marked as black crosses if provided.

    Returns:
    - fig (Figure): The Plotly figure object for the plot.
    """

    # Create the base cluster plot
    fig = px.scatter(
        pca_df,
        x='x',
        y='y',
        opacity=0.6,
        size_max=4,
        color= pca_df.cluster.astype(str),
        title='2D Visualization of Clusters with Recently Viewed Products',
        labels={'x': 'PCA Component 1', 'y': 'PCA Component 2'},
        category_orders={'cluster': list(range(num_clusters))},
        # show the product id in the tooltip
        hover_data={'product_id': True}

    )

    # Update layout to add legend title and adjust plot settings
    fig.update_layout(legend_title_text='Clusters', legend=dict(x=1, y=1), width=600, height=500)

    return fig

fig = plot_clusters(pca_df)
fig.show()

### Task 5 - Highlight recently viewed products

In [63]:
print("The user has recently viewed the following products: ", recently_viewed_products)

The user has recently viewed the following products:  ['P316', 'P333', 'P1115', 'P1691', 'P1082', 'P397', 'P1441', 'P1054']


Let's have a look at the records in our `clustered_data` dataframe related to the recently viewed products.

In [65]:
filtered_data = clustered_data.where(F.col("product_id").isin(recently_viewed_products))
unique_clusters = filtered_data.select("cluster").distinct().rdd.flatMap(lambda x: x).collect()
unique_clusters

[2, 3]

### Task 6 - Recommend products based on recently viewed products

Let's have a look at the recently viewed products titles

In [66]:
filtered_data.select("title").rdd.flatMap(lambda x: x).collect()

["Mystic Fitz Roy Beige 9' 0 x 12' 0 Area Rug",
 'Florida Shag Beige/Multi 3 ft. x 5 ft. Floral Area Rug',
 '1 gal. #M250-3 Apple Turnover Extra Durable Flat Interior Paint & Primer',
 '1 gal. #HDPG60 Misty Emerald Lake Flat Interior Paint and Primer',
 '1 qt. #S220-7 Molasses Extra Durable Flat Interior Paint & Primer',
 'Modern Gray/Multi 9 ft. x 12 ft. Vibrant Abstract Polyester Area Rug',
 '1 qt. #PPU6-06 Honey Locust Eggshell Enamel Low Odor Interior Paint & Primer',
 'Genet Rust/Red-Brown 8 ft. x 11 ft. Abstract Wool Area Rug']

Let's see the distinct clusters of the recenetly viewed products.

In [67]:
print(unique_clusters)

[2, 3]


Let's find the possible products for the recommendation.

In [70]:
possible_recommendations=clustered_data.filter(F.col("cluster").isin(unique_clusters)).filter(clustered_data.product_id.isin(recently_viewed_products))

Let's perform a groupby and generate a list of product IDs that can be recommended for each of the clusters.

In [75]:
recommendations = possible_recommendations.groupBy("cluster").agg(F.collect_list("product_id").alias("recommendations"))
recommendations_df = recommendations.toPandas()
recommendations_df["random_recommendations"] = recommendations_df.recommendations.apply(lambda x: np.random.choice(x, min(5, len(x)), replace =False).tolist()) # Change here to sample at most the length of x
recommendations_df.head()

,cluster,recommendations,random_recommendations
0,2,"[P316, P333, P1115, P1691]","[P1115, P333, P1691, P316]"
1,3,"[P397, P1054, P1082, P1441]","[P1441, P1082, P1054, P397]"


In [76]:
# write a python function to display the recommendations
def display_recommendations(row):
  # find the title of the product in df
  product_ids = row['random_recommendations']
  cluster = row.cluster

  titles = data. \
          filter(data["product_id"]. \
          isin(product_ids)).select("title").collect()

  print("\n")
  print("Recommendations for Cluster:", cluster)
  for title in titles:
    print(title[0])

recommendations_df.apply(display_recommendations, axis=1)



Recommendations for Cluster: 2
Mystic Fitz Roy Beige 9' 0 x 12' 0 Area Rug
Florida Shag Beige/Multi 3 ft. x 5 ft. Floral Area Rug
Modern Gray/Multi 9 ft. x 12 ft. Vibrant Abstract Polyester Area Rug
Genet Rust/Red-Brown 8 ft. x 11 ft. Abstract Wool Area Rug


Recommendations for Cluster: 3
1 gal. #M250-3 Apple Turnover Extra Durable Flat Interior Paint & Primer
1 gal. #HDPG60 Misty Emerald Lake Flat Interior Paint and Primer
1 qt. #S220-7 Molasses Extra Durable Flat Interior Paint & Primer
1 qt. #PPU6-06 Honey Locust Eggshell Enamel Low Odor Interior Paint & Primer


,0
0,None
1,None
